In [1]:
print("    Version control\n------------------------")
import os     # operating system interfaces
import fnmatch
import random
import datetime
import numpy as np;              print("Numpy\t\t", np.__version__)
import matplotlib as mpl;        print("matplotlib\t", mpl.__version__)
import matplotlib.pyplot as plt
import nibabel as nib;           print("NiBabel\t\t {}".format(nib.__version__))
from nibabel.testing import data_path
import math
import pandas as pd;             print("Pandas\t\t {}".format(pd.__version__))
import sys
import imageio;                  print("imageio\t\t {}".format(imageio.__version__))
import h5py;                     print("H5py\t\t {}".format(h5py.__version__))
import sklearn;                  print("Scikit-learn\t {}".format(sklearn.__version__))
import skimage;                  print("Scikit-image\t {}".format(skimage.__version__))
import tensorflow as tf;         print("TensorFlow\t {}".format(tf.dr__version__))
import keras as K;               print("Keras\t\t {}".format(K.__version__))
from tensorflow.keras import models, Input, Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, BatchNormalization, Conv3D, MaxPooling3D, UpSampling3D
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import TensorBoard, EarlyStopping

    Version control
------------------------
Numpy		 1.19.4
matplotlib	 3.3.3
NiBabel		 3.2.0
Pandas		 1.1.4
imageio		 2.9.0
H5py		 2.10.0
Scikit-learn	 0.23.2
Scikit-image	 0.17.2
TensorFlow	 2.3.1
Keras		 2.3.1


Using TensorFlow backend.


In [2]:
## Loading dataset from path directory
print(os.getcwd())     # '/Users/shossein/GitHub/DTI_Clustering'

## load train data
sample_train_subset = np.loadtxt("train_100_sample_name.csv", dtype=str, delimiter=",")
train_data = np.load('train_100.npy').reshape(100,182,218,182,1)
print('train_data shape is {}'.format(train_data.shape))

## load validation data
sample_val_subset = np.loadtxt("valid_24_sample_name.csv", dtype=str, delimiter=",")
valid_data = np.load('valid_24.npy').reshape(24,182,218,182,1)
print('valid_data shape is {}'.format(valid_data.shape))

/Users/shossein/GitHub/DTI_Clustering
train_data shape is (100, 182, 218, 182, 1)
valid_data shape is (24, 182, 218, 182, 1)


In [8]:
## Define parameters:
IMAGE_HEIGHT = train_data.shape[1]
IMAGE_WIDTH = train_data.shape[2]
IMAGE_DEPTH = train_data.shape[3]
batch_size = 1
data_shape = [1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
input_shape = [batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
print("input-layer shape:", input_shape)

## Encoder
input_img = Input(shape=(182, 218, 182, 1))
x = Conv3D(filters=1, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv1')(input_img)
x = BatchNormalization()(x)
# x = MaxPooling3D(pool_size=(2, 2, 2), padding='valid')(x)

x = Conv3D(filters=1, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv2')(x)
x = BatchNormalization()(x)
# x = MaxPooling3D(pool_size=(2, 2, 2), padding='valid')(x)

x = Conv3D(filters=1, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv3')(x)
x = BatchNormalization()(x)
# x = MaxPooling3D(pool_size=(2, 2, 2), padding='valid')(x)

## Decoder
x = Conv3D(filters=1, kernel_size=3, padding='same', activation='relu', name='DeConv1')(x)
x = BatchNormalization()(x)
x = UpSampling3D(size=(2, 2, 2))(x)

x = Conv3D(filters=1, kernel_size=3, padding='same', activation='relu', name='DeConv2')(x)
x = BatchNormalization()(x)
x = UpSampling3D(size=(2, 2, 2))(x)

x = Conv3D(filters=1, kernel_size=(2,4,2), padding='valid', activation='relu', name='DeConv3')(x)
x = BatchNormalization()(x)
x = UpSampling3D(size=(2, 2, 2))(x)

decoded = Conv3D(filters=1, kernel_size=3, padding='same', activation='sigmoid', name='Output')(x)

model_CAE = Model(inputs=input_img, outputs=decoded)
## optimizer=rmsprop, sgd    loss=binary_crossentropy, SparseCategoricalCrossentropy(from_logits=False)
model_CAE.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model_CAE.summary()

input-layer shape: [1, 182, 218, 182, 1]
Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 182, 218, 182, 1) 0         
_________________________________________________________________
Conv1 (Conv3D)               (None, 91, 109, 91, 1)    28        
_________________________________________________________________
batch_normalization_12 (Batc (None, 91, 109, 91, 1)    4         
_________________________________________________________________
Conv2 (Conv3D)               (None, 46, 55, 46, 1)     28        
_________________________________________________________________
batch_normalization_13 (Batc (None, 46, 55, 46, 1)     4         
_________________________________________________________________
Conv3 (Conv3D)               (None, 23, 28, 23, 1)     28        
_________________________________________________________________
batch_normali

In [6]:
## Tensorboard
logdir = os.path.join("CAE_logs", datetime.datetime.now().strftime("%Y_%m_%d____%H_%M_%S"))
tb_callback = TensorBoard(logdir, histogram_freq=1)

In [9]:
## Start time:
from datetime import datetime
start_time = datetime.now().strftime("%H:%M:%S")
print("\nStart Time =", start_time, "\n")

## Midel Fit
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=5, mode='auto')
model_CAE.fit(train_data, train_data, epochs=50, batch_size=batch_size, shuffle=True, validation_data=(valid_data, valid_data), callbacks=[tb_callback], verbose=1)
model_CAE.save_weights("CAE_weights.hdf5")
# autoencoder.load_weights("CAE_weights.hdf5")     # loading weights

# End time:
from datetime import datetime
end_time = datetime.now().strftime("%H:%M:%S")
print("\nEnd Time =", end_time)


Start Time = 22:51:01 

Epoch 1/50
100/100 [==============================] - 526s 5s/step - loss: 0.1413 - accuracy: 0.6499 - val_loss: 0.0516 - val_accuracy: 0.7056
Epoch 2/50
100/100 [==============================] - 488s 5s/step - loss: 0.0562 - accuracy: 0.7005 - val_loss: 0.0168 - val_accuracy: 0.7067
Epoch 3/50
100/100 [==============================] - 444s 4s/step - loss: 0.0308 - accuracy: 0.7018 - val_loss: 0.0150 - val_accuracy: 0.7067
Epoch 4/50
100/100 [==============================] - 476s 5s/step - loss: 0.0183 - accuracy: 0.7023 - val_loss: 0.0151 - val_accuracy: 0.7067
Epoch 5/50
100/100 [==============================] - 472s 5s/step - loss: 0.0155 - accuracy: 0.7023 - val_loss: 0.0151 - val_accuracy: 0.7067
Epoch 6/50
100/100 [==============================] - 420s 4s/step - loss: 0.0151 - accuracy: 0.7023 - val_loss: 0.0150 - val_accuracy: 0.7067
Epoch 7/50
100/100 [==============================] - 417s 4s/step - loss: 0.0150 - accuracy: 0.7023 - val_loss: 0.01

OSError: Unable to create file (unable to truncate a file which is already open)

In [ ]:
model_CAE.load_weights("CAE_weights.hdf5")     # loading weights
test_data = train_data[0,:].reshape(1, 182, 218, 182, 1)
reconstructed = model_CAE.predict(test_data)

print('\ntrain_data[0,100,100:110,100]\n\n {}'.format(train_data[0,100,100:110,100]),'\n')
print('\nReconstructed_data[0,100,100:110,100]\n\n {}'.format(reconstructed[0,100,100:110,100]),'\n')

In [5]:
h5_file = h5py.File('CAE_weights.hdf5', 'r')
print(list(h5_file.keys()))
# for L in (list(h5_file.keys())):
#     print(L)
#     L = h5_file[layer]
#     W = L[layer]['kernel:0']
#     print(W.shape)

L = h5_file[list(h5_file.keys())[0]]
W = L[list(h5_file.keys())[0]]['kernel:0']
print('\nWeights shape: {}'.format(W.shape))
print('\nWeights[1][1][1]: {}'.format(W[1][1][1]))

['Conv1', 'Conv2', 'Conv3', 'DeConv1', 'DeConv2', 'DeConv3', 'Output', 'batch_normalization', 'batch_normalization_1', 'batch_normalization_2', 'batch_normalization_3', 'batch_normalization_4', 'batch_normalization_5', 'input_1', 'up_sampling3d', 'up_sampling3d_1', 'up_sampling3d_2']

Weights shape: (3, 3, 3, 1, 64)

Weights[1][1][1]: [[ 0.0037242   0.02097484  0.01185212  0.04621937  0.0556117  -0.02928501
  -0.02970716 -0.01357271  0.02839159  0.03565723 -0.02163431  0.03573501
   0.04866397  0.0811149   0.03360678  0.02316148  0.01399892 -0.04551153
  -0.01990886  0.02220522  0.06904128 -0.03217261  0.02329263  0.01268102
  -0.03980312  0.03758161 -0.03650775 -0.03783716 -0.03376658 -0.02598745
  -0.05506312  0.03708598 -0.02442701  0.02998579  0.01478631 -0.05314519
  -0.02072836  0.04768707 -0.03685523 -0.0557698   0.04025914 -0.06789807
  -0.03865194  0.00365022  0.0402042   0.05837304  0.07352968  0.02735536
   0.00031396  0.03972595  0.00384991 -0.05485379 -0.0627759  -0.037174